In [1]:
import tensorflow as tf
from lbn import LBN, LBNLayer
import numpy as np

In [2]:
lbn = LBN(10, boost_mode=LBN.PAIRS)

In [3]:
def create_four_vectors(n, p_low=-100., p_high=100., m_low=0.1, m_high=50.):
    """
    Creates a numpy array with shape ``n + (4,)`` describing four-vectors of particles whose
    momentum components are uniformly distributed between *p_low* and *p_high*, and masses between
    *m_low* and *m_high*.
    """
    # create random four-vectors
    if not isinstance(n, tuple):
        n = (n,)
    vecs = np.random.uniform(p_low, p_high, n + (4,)).astype(np.float32)

    # the energy is also random and might be lower than the momentum,
    # so draw uniformly distributed masses, and compute and insert the energy
    m = np.abs(np.random.uniform(m_low, m_high, n))
    p = np.sqrt(np.sum(vecs[..., 1:]**2, axis=-1))
    E = (p**2 + m**2)**0.5
    vecs[..., 0] = E

    return vecs

In [4]:
inputs = create_four_vectors((1, 10))
inputs.shape

(1, 10, 4)

In [5]:
inputs

array([[[ 9.27365875e+01,  6.18353996e+01,  5.54067993e+01,
         -7.88727474e+00],
        [ 7.65168457e+01, -7.60194168e+01, -1.30037144e-01,
          7.75416946e+00],
        [ 8.97892151e+01,  8.34566040e+01,  2.83305874e+01,
         -1.37429199e+01],
        [ 1.14965729e+02,  8.38319702e+01,  3.92403183e+01,
          6.36488800e+01],
        [ 1.06519585e+02,  5.32300987e+01,  3.75962257e+01,
          7.78144531e+01],
        [ 8.09133530e+01, -4.57087898e+01, -2.97864079e+00,
          4.54836082e+01],
        [ 1.47624023e+02,  9.38040619e+01, -8.49045410e+01,
         -6.68634796e+01],
        [ 7.66401062e+01,  7.09525452e+01,  2.27315445e+01,
          8.83832741e+00],
        [ 7.46975479e+01, -4.32757683e+01, -3.69345741e+01,
          3.86231461e+01],
        [ 4.83133888e+01,  2.62829393e-01,  3.96751556e+01,
         -2.14659004e+01]]], dtype=float32)

In [6]:
lbn.available_features

['E',
 'beta',
 'eta',
 'gamma',
 'm',
 'p',
 'pair_cos',
 'pair_dr',
 'pair_ds',
 'pair_dy',
 'phi',
 'pt',
 'px',
 'py',
 'pz']

In [7]:
lbn.build(inputs.shape, features=["E", "pt", "eta", "phi", "m"])

2022-03-25 13:30:00.423148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 13:30:00.465651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 13:30:00.465853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 13:30:00.466802: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
features = lbn(inputs)

In [9]:
features.shape

TensorShape([1, 50])

In [10]:
features # 10 x (E, pt, eta, phi, m)

<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[ 8.6944962e+01,  8.8801735e+01,  6.4248955e+01,  6.9607880e+01,
         5.9394951e+01,  6.5790573e+01,  7.5689133e+01,  9.7649734e+01,
         7.4891670e+01,  6.5334641e+01,  3.3295345e+01,  2.9684593e+01,
         7.8342266e+00,  1.2541226e+01,  8.9554834e+00,  1.7659157e+01,
         1.7537893e+01,  2.9303848e+01,  1.7879885e+01,  1.9114227e+01,
        -3.8208090e-02, -1.8515429e-01,  1.0186713e+00, -2.8407407e-01,
         1.7880819e+00, -2.5318113e-01,  4.2220843e-01,  1.1699503e-02,
        -1.0366693e+00, -4.4616818e-01, -1.9929755e+00,  4.1621976e-02,
         1.4174333e+00,  2.1321025e-01,  2.5401661e+00, -2.4974683e-01,
         3.0001390e+00, -2.5175421e+00, -1.0159320e+00,  1.5541777e-01,
         8.0307083e+01,  8.3510590e+01,  6.3067822e+01,  6.8373512e+01,
         5.2636322e+01,  6.3214977e+01,  7.3233208e+01,  9.3148476e+01,
         6.9306389e+01,  6.1851246e+01]], dtype=float32)>

In [12]:
lbn.particles

<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 83.6875    ,  20.505634  , -11.003374  ,   3.567997  ],
        [ 99.00972   ,  43.919838  ,  18.525673  ,  23.593699  ],
        [ 77.766235  ,  37.228622  ,  15.324515  ,  21.193653  ],
        [ 73.241776  ,  24.577984  ,   4.421159  ,   8.108311  ],
        [ 61.645515  ,  23.820217  ,  13.987604  ,  16.324379  ],
        [ 73.24385   ,  35.92362   ,   4.6210914 ,   7.524593  ],
        [ 78.95617   ,   1.9152074 ,  12.959745  ,  26.444023  ],
        [ 94.299286  ,  14.533326  ,  -2.0024834 ,  -0.62019205],
        [ 85.77142   ,  50.087196  ,   6.0044017 ,   2.9148316 ],
        [ 70.26694   ,  32.179504  ,   8.16977   ,   3.0910861 ]]],
      dtype=float32)>

In [13]:
lbn.restframes

<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 90.75711   ,  34.473526  ,  19.553003  ,   4.8865476 ],
        [ 79.18158   ,  11.581448  ,  14.041772  ,  23.649742  ],
        [ 74.16606   ,  35.038757  ,   7.3732634 ,  11.435421  ],
        [114.75878   ,  19.5162    ,   2.7996693 ,  18.563143  ],
        [ 86.306526  ,  41.26809   ,  11.7973795 , -12.820984  ],
        [ 46.212494  ,  12.141587  ,   5.7994237 ,   7.7729273 ],
        [ 87.32028   ,  21.035408  ,  11.443278  ,  20.53291   ],
        [ 64.432556  ,  24.589224  ,   9.704107  ,  -0.61806273],
        [ 76.613525  ,  35.080467  ,  18.288998  ,  21.525145  ],
        [ 71.7521    ,  13.81082   ,   5.412933  ,  12.366296  ]]],
      dtype=float32)>

In [14]:
lbn.boosted_particles

<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 86.94496  , -13.642741 , -30.371954 ,  -1.2724609],
        [ 88.801735 ,  29.658884 ,   1.2351747,  -5.527687 ],
        [ 64.248955 ,   1.1967776,   7.7422757,   9.434125 ],
        [ 69.60788  ,  12.257252 ,   2.6537054,  -3.6107473],
        [ 59.39495  ,  -7.384058 ,   5.0671844,  26.018784 ],
        [ 65.79057  ,  17.111282 ,  -4.3646135,  -4.5188837],
        [ 75.68913  , -17.362726 ,   2.4725342,   7.626608 ],
        [ 97.649734 , -23.780607 , -17.123034 ,   0.3428483],
        [ 74.89167  ,   9.419624 , -15.197399 , -22.038525 ],
        [ 65.33464  ,  18.883844 ,   2.9587457,  -8.813934 ]]],
      dtype=float32)>

In [15]:
def mass(x):
    return tf.sqrt(x[:,:,0]**2 - x[:,:,1]**2 - x[:,:,2]**2 - x[:,:,3]**2)

In [16]:
mass(lbn.particles)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[80.3076  , 83.51113 , 63.068264, 68.37389 , 52.636757, 63.21536 ,
        73.23365 , 93.14904 , 69.307   , 61.851612]], dtype=float32)>

In [17]:
mass(lbn.boosted_particles)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[80.30708 , 83.51058 , 63.067825, 68.37351 , 52.636322, 63.214977,
        73.2332  , 93.148476, 69.30639 , 61.851242]], dtype=float32)>

In [18]:
mass(inputs)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[40.550404 ,  3.9663491, 10.276581 , 24.460932 , 32.31738  ,
        48.78572  , 36.25111  , 15.639795 , 29.172274 , 17.297737 ]],
      dtype=float32)>

In [18]:
mass(lbn.restframes)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[95.082405, 89.21855 , 62.378822, 36.677788, 58.693535, 75.2365  ,
        69.49269 , 56.932716, 78.61988 , 44.61713 ]], dtype=float32)>